<a href="https://colab.research.google.com/github/cij1012/3-min-pytorch/blob/main/4-4%20%EA%B3%BC%EC%A0%81%ED%95%A9%EA%B3%BC%20%EB%93%9C%EB%A1%AD%EC%95%84%EC%9B%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [3]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

In [4]:
Epochs = 50
Batch_size = 64

In [10]:
# 데이터셋에 노이즈 추가하기

train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                          train=True,
                          download=True,
                   transform=transforms.Compose([
                       transforms.RandomHorizontalFlip(), # 이미지를 무작위로 수평뒤집기
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
                          batch_size = Batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=Batch_size, shuffle=True)

In [13]:
# 뉴럴넷으로 FMNIST 학습하기

class Net(nn.Module):
  def __init__(self, dropout_p=0.2): # dropout_p만큼 날리고 사용
    super(Net, self).__init__()
    self.fc1 = nn.Linear(784, 256)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 10)
    self.dropout_p = dropout_p

  def forward(self, x): # 각 층마다 dropout 시행
    x = x.view(-1, 784)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training,
                  p=self.dropout_p)
    x = F.relu(self.fc2(x))
    x = F.dropout(x, training=self.training,
                  p=self.dropout_p)
    x = self.fc3(x)
    return x

In [14]:
model = Net(dropout_p=0.2).to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [15]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [16]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()
            
            # 맞춘 갯수 계산
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [18]:
for epoch in range(1, Epochs + 1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

[1] Test Loss: 0.6488, Accuracy: 77.75%
[2] Test Loss: 0.5306, Accuracy: 80.98%
[3] Test Loss: 0.4893, Accuracy: 82.55%
[4] Test Loss: 0.4593, Accuracy: 83.28%
[5] Test Loss: 0.4377, Accuracy: 84.03%
[6] Test Loss: 0.4289, Accuracy: 84.55%
[7] Test Loss: 0.4118, Accuracy: 85.08%
[8] Test Loss: 0.4104, Accuracy: 84.91%
[9] Test Loss: 0.4019, Accuracy: 85.70%
[10] Test Loss: 0.3923, Accuracy: 85.51%
[11] Test Loss: 0.3845, Accuracy: 86.17%
[12] Test Loss: 0.3846, Accuracy: 86.23%
[13] Test Loss: 0.3731, Accuracy: 86.62%
[14] Test Loss: 0.3700, Accuracy: 86.53%
[15] Test Loss: 0.3661, Accuracy: 86.80%
[16] Test Loss: 0.3640, Accuracy: 86.76%
[17] Test Loss: 0.3581, Accuracy: 87.03%
[18] Test Loss: 0.3554, Accuracy: 87.02%
[19] Test Loss: 0.3509, Accuracy: 87.21%
[20] Test Loss: 0.3499, Accuracy: 87.28%
[21] Test Loss: 0.3529, Accuracy: 87.22%
[22] Test Loss: 0.3464, Accuracy: 87.52%
[23] Test Loss: 0.3465, Accuracy: 87.51%
[24] Test Loss: 0.3386, Accuracy: 87.62%
[25] Test Loss: 0.3390, A